## Data Preprocessing

In [11]:

import numpy as np
import pandas as pd
import math

data = pd.read_csv('titanic.csv')
data = data.sample(frac=1)
x = data[["Age", "Pclass", "Fare"]].to_numpy()
y = data["Survived"].to_numpy()
datalen = len(x)
split_percent = 0.05    #10% used for testing data 
split = math.floor(datalen*split_percent)

#no need to shuffle, data already randomized in the .csv file
x_test = x[:split].T
x_train = x[split:].T
y_test = y[:split]
y_train = y[split:]

#print(x_test)


In [15]:
#This is NOT my submission for task 3
#This is just to double check my implementation's results against an offical implementation

from sklearn.linear_model import LogisticRegression
m = LogisticRegression(penalty="none", solver="sag")
m.fit(x_train.T, y_train)
trainacc = m.score(x_train.T, y_train)
testacc = m.score(x_test.T, y_test)
print(f"TrainAcc: {trainacc:0.2f}, TestAcc: {testacc:0.2f}")

TrainAcc: 0.67, TestAcc: 0.75


/home/james/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## 

In [16]:

#hyperparams
epochs = 100
learning_rate = 0.003

#init our weight vector based on how many features we have in the data
weights = np.zeros(x.shape[1])
bias = 0

#compute the sigmoid of the input X
#when X is an array, returns the array with sigmoid applied elementwise
def sigmoid(X):
    return 1/(1+np.exp(-X))

# X: Model input
def forward(X, weights, bias):
    return sigmoid(np.dot(weights.T, X) + bias)

#compute individual loss for each sample (the negitive has been factored out into the cost)
#(in this case array of individual losses for each sample)
# Y: Labels, A: Model Outputs
def loss(Y, A):
    return Y*np.log(A) + (1-Y)*np.log(1-A)

#Compute cost
# Y: Labels, A: Model Outputs
def cost(Y, A):
    return -1/len(A) * np.sum(loss(Y, A))

#partial derivitive of cost with respect to weights
def dCostWRTw(X, A, Y):
    return np.dot(X, (A-Y).T)/len(A)

#partial derivitive of cost with respect to bias
def dCostWRTb(A, Y):
    return np.sum(A-Y)/len(A)

#Predict the 
# X: Model input
def predict(X, weights, bias):
    A = forward(X, weights, bias)
    # Convert the entries of a into 0 (if activation <= 0.5) or
    # 1 (if activation > 0.5) and store the predictions in a vector.
    p = np.copy(A)
    p[p <= 0.5] = 0
    p[p > 0.5] = 1
    return p 

def accuracy(predictions, labels):
    return np.count_nonzero(predictions==labels)/len(predictions)

#training loop
for epoch in range(epochs):
    A = forward(x_train, weights, bias)
    c = cost(y_train, A)
    
    #compute grads of the cost func
    dw = dCostWRTw(x_train, A, y_train)
    db = dCostWRTb(A, y_train)

    #update the weights + bias with respect to our grads scaled by our learning rate hyperparm
    weights = weights - learning_rate * dw
    bias = bias - learning_rate * db

#Predict results and compute accuracy
trainPredictions = predict(x_train, weights, bias)
trainacc = accuracy(trainPredictions, y_train)
testPredictions = predict(x_test, weights, bias)
testacc = accuracy(testPredictions, y_test) 
print(f"TrainAcc: {trainacc:0.2f}, TestAcc: {testacc:0.2f}")




Epoch: 99, Cost: 0.63, TrainAcc: 0.65, TestAcc: 0.75
